# MCP Server & Client Implementation Using Fastmcp 

---
## Architecture

LLM Host (MCP Client)  <—— JSON-RPC ——>  fastmcp Server  <——> CSV Dataset


We'll:
1. Launch a server exposing two tools (`summarize`, `query`).
2. Use an MCP Client to discover and call those tools.
3. Simulate a host (agent) deciding which tool to call.
---
## Start the MCP Server

In a separate terminal, run:
```bash
python mcp_server_fastmcp.py
```
This starts an MCP-compliant server exposing `summarize` and `query` tools.

---


### 1. Install dependencies

In [2]:

# %pip install fastmcp pandas langchain openai uvicorn --upgrade --no-cache-dir

## Connect via MCP Client

Once the server is running, connect using the MCP client to list tools and invoke them.

In [28]:
from fastmcp import Client
from fastmcp.client import PythonStdioTransport
import asyncio

async def main():
    transport = PythonStdioTransport("mcp_server_fastmcp.py")
    async with Client(transport) as client:
        tools = await client.list_tools()
        print("Tools:", [t.name for t in tools])

await main()


Tools: ['summarize', 'query']


## Simulate a Simple Host Agent
Let's simulate a rule-based 'AI agent' that decides whether to use `summarize` or `query` based on user text.

In [29]:
def decide_tool(text: str):
    text = text.lower()
    if "summarize" in text or "overview" in text:
        return "summarize", {}
    if "west" in text:
        return "query", {"expr": "region == 'West' and sales > 1500"}
    return "summarize", {}

async def run_agent(user_input, client):
    tool, params = decide_tool(user_input)
    print(f"Agent decided to use '{tool}'")

    # API for fastmcp 2.12.5
    result = await client.call_tool(tool, params)

    print("Result:", result, "\n")



In [30]:
# create a connection to your MCP server
transport = PythonStdioTransport("mcp_server_fastmcp.py")

async with Client(transport) as client:
    await run_agent("Give me a summary of the dataset", client)
    await run_agent("Show West region sales > 1500", client)

Agent decided to use 'summarize'
Result: CallToolResult(content=[TextContent(type='text', text='{"rows":10,"columns":["order_id","region","sales","quarter","rep"],"numeric_stats":{"order_id":{"count":10.0,"unique":"","top":"","freq":"","mean":105.5,"std":3.0276503540974917,"min":101.0,"25%":103.25,"50%":105.5,"75%":107.75,"max":110.0},"region":{"count":10,"unique":4,"top":"West","freq":4,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""},"sales":{"count":10.0,"unique":"","top":"","freq":"","mean":1235.0,"std":536.4751210965477,"min":600.0,"25%":825.0,"50%":1175.0,"75%":1450.0,"max":2200.0},"quarter":{"count":10,"unique":4,"top":"Q1","freq":3,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""},"rep":{"count":10,"unique":10,"top":"Alex","freq":1,"mean":"","std":"","min":"","25%":"","50%":"","75%":"","max":""}}}', annotations=None, meta=None)], structured_content={'rows': 10, 'columns': ['order_id', 'region', 'sales', 'quarter', 'rep'], 'numeric_stats': {'order_

---
##  Architecture Recap
```
+--------------------+      JSON-RPC (MCP)      +--------------------+
|  Host / AI Agent   |  <-------------------->  | fastmcp MCP Server |
| (LLM or Router)    |                        | (summarize, query)  |
+--------------------+                         +--------------------+
```
---